In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from glob import glob
import re
import ast
import cv2
import csv
import time
import ast
import urllib
from PIL import Image, ImageDraw
from tqdm import tqdm
from dask import bag, threaded
import matplotlib
import matplotlib.pyplot as pltc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from dask import bag, threaded
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.applications import MobileNet
from collections import Counter 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
BASE_SIZE = 256
DP_DIR = "../input/shufflecsvs/shufflecsv/"
INPUT_DIR = '../input/quickdraw-doodle-recognition/'
TEST_DIR="../input/shuffletestcsv/shuffletestcsvs/"
NCSVS = 4
NCATS = 340
import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

['shufflecsvs', 'quickdraw-doodle-recognition', 'shuffletestcsv', 'keraspretrainedmodels']


In [2]:
print(os.listdir("../input/shufflecsvs/shufflecsv"))
print(os.listdir("../input/shuffletestcsv/shuffletestcsvs"))

['train_k2.csv.gz', 'train_k3.csv.gz', 'train_k0.csv.gz', 'train_k1.csv.gz']
['test_k0.csv.gz', 'test_k1.csv.gz', 'test_k2.csv.gz', 'test_k3.csv.gz']


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))
count=0
count=len(pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/apple.csv"))

print("count: ", count)
print(os.listdir("../input"))
df_arr=pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/apple.csv", iterator=True, chunksize=count//1000)
print("df_arr: ",df_arr)
df=pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/apple.csv")
print("df.head(): ",df.head())
print("df.columns: ", df.columns)
print("printing df_arr ----- ")
ii=0
for i in range(count//1000):
    print("i: ",i," : ", len(pd.DataFrame(df_arr.get_chunk(count//1000)) ))


count:  144722
['shufflecsvs', 'quickdraw-doodle-recognition', 'shuffletestcsv', 'keraspretrainedmodels']
df_arr:  <pandas.io.parsers.TextFileReader object at 0x7f1d2376bda0>
df.head():    countrycode  ...     word
0          US  ...    apple
1          RU  ...    apple
2          GB  ...    apple
3          US  ...    apple
4          TH  ...    apple

[5 rows x 6 columns]
df.columns:  Index(['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word'], dtype='object')
printing df_arr ----- 
i:  0  :  144
i:  1  :  144
i:  2  :  144
i:  3  :  144
i:  4  :  144
i:  5  :  144
i:  6  :  144
i:  7  :  144
i:  8  :  144
i:  9  :  144
i:  10  :  144
i:  11  :  144
i:  12  :  144
i:  13  :  144
i:  14  :  144
i:  15  :  144
i:  16  :  144
i:  17  :  144
i:  18  :  144
i:  19  :  144
i:  20  :  144
i:  21  :  144
i:  22  :  144
i:  23  :  144
i:  24  :  144
i:  25  :  144
i:  26  :  144
i:  27  :  144
i:  28  :  144
i:  29  :  144
i:  30  :  144
i:  31  :  144
i:  32  :  144
i:  33 

In [4]:
#messageData.py modified
import numpy as np
import os
import random
import pandas as pd
class messageData:
    def __init__(self):
        self.total_len_train=0
        self.total_len_valid=0
        self.files=['apple.csv', 'banana.csv', 'circle.csv', 'pineapple.csv']
        self.class_to_idx = {v.split('.')[0]: k for k, v in enumerate(self.files)}
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
    def setLengths(self, folder=DP_DIR, ks=4):
        self.total_len_train=0
        self.total_len_valid=0
        self.folder=folder
        flag=True
        sum1=0
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            name = filename.split('.')[0]
            samples=pd.read_csv(filename)#np.genfromtxt(fullpath, delimiter=',')
            print(type(samples))
            sum1=sum1+samples['drawing'].shape[0]
            s_temp=int(samples.shape[0])
            self.total_len_train+= s_temp*0.7
            self.total_len_valid+= s_temp*0.3
        print("sum1: ", sum1)
        self.sum1 = sum1
    
    def draw_cv2(self, raw_strokes, size=256, lw=6):
        img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
        for stroke in raw_strokes:
            for i in range(len(stroke[0]) - 1):
                _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
        if size != BASE_SIZE:
            return cv2.resize(img, (size, size))
        else:
            return img
#----------------------------------------------------------------------------------------------------------------------------------------------------
    def generator(self, folder=DP_DIR, batch_size=128, size=80, lw=6, num_classes=4, ks=4):
        while True:
            for k in np.random.permutation(ks):
                filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
                for df in pd.read_csv(filename, chunksize=batch_size):
                    df['drawing'] = df['drawing'].apply(ast.literal_eval)
                    x = np.zeros((len(df), size, size))
                    for i, raw_strokes in enumerate(df.drawing.values):
                        x[i] = self.draw_cv2(raw_strokes, size=size, lw=lw)
                    x = x / 255.
                    x = x.reshape((len(df), size*size)).astype(np.float32)
                    y = tf.keras.utils.to_categorical(df.y, num_classes=num_classes)
                    yield x, y
                    
    def df_to_image_array(self, df, size=80, lw=6):
        df['drawing'] = df['drawing'].apply(ast.literal_eval)
        x = np.zeros((len(df), size, size))
        for i, raw_strokes in enumerate(df.drawing.values):
            x[i] = self.draw_cv2(raw_strokes, size=size, lw=lw)
        x = x / 255.
        x = x.reshape((len(df), size*size)).astype(np.float32)
        return x
    def makeValidationData(self, TEST_DIR=TEST_DIR, NCSVS=4, NCATS=4, size=80):
        valid_df=pd.read_csv(os.path.join(TEST_DIR, 'test_k{}.csv.gz'.format(0)) )
        self.x_valid = self.df_to_image_array(valid_df, size)
        self.y_valid = tf.keras.utils.to_categorical(valid_df.y, num_classes=NCATS)
        print("validation data dimensions: ", self.x_valid.shape[0], " , ", self.y_valid.shape)
        
data=messageData()
data.setLengths()
data.makeValidationData()
print("total_len_train", data.total_len_train)
print("total_len_valid", data.total_len_valid)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
sum1:  120000
validation data dimensions:  10114  ,  (10114, 4)
total_len_train 84000.0
total_len_valid 36000.0


In [5]:
#baseline.py modified
import numpy
import time
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Input
from keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.callbacks import TensorBoard
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.optimizers import Adam
#import massageData

CLASS_NUM = 4
BITMAP_DIM=80*80
BATCH_SIZE=128
EPOCHS=2
STEPS = 1000
model=None
def baseline_model():
    # create model
    #model=Sequential()
    #model.add(Dense(1, input_dim=BITMAP_DIM, activation='sigmoid'))
    #model.add(Dense(CLASS_NUM, activation='softmax'))
    input_X = Input(shape=(BITMAP_DIM,))
    x = Dense(100)(input_X)
    x = Dense(CLASS_NUM, activation="softmax")(x)
    model = Model(inputs=input_X, outputs=x)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=5e-5), metrics=['accuracy'])
    return model

def main():
    print ("Done loading the libraries")
    t0 = time.time()
    seed = 7
    numpy.random.seed(seed)
    data = messageData()
    data.setLengths()
    data.makeValidationData()
    #generators
    #train_generator = data.generator(folder=DP_DIR, batch_size=BATCH_SIZE) 
    train_generator=data.generator( batch_size=BATCH_SIZE, ks=range(NCSVS - 1))
    #validation_generator = data.generator('../input/train_simplified/', isTrain=False, batch_size=BATCH_SIZE)
    print("length of train_samples: ",data.total_len_train)
    print("length of validation_samples: ",data.total_len_valid)
    model = baseline_model()
    print(model.summary())
    try:
        model.fit_generator(train_generator, samples_per_epoch=data.sum1, validation_data=(data.x_valid,data.y_valid), 
                            nb_val_samples=0, epochs=3, steps_per_epoch=data.sum1/BATCH_SIZE, verbose=1, ) #steps_per_epoch=STEPS
    except KeyboardInterrupt as e:
        print(str(e))
    print("Model training, testing completed- Saving Model...")
    model.save("doodle_trial_model.h5")
if __name__ == '__main__':
    main()
"""
Done loading the libraries
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
sum1:  120000
validation data dimensions:  10114  ,  (10114, 4)
length of train_samples:  84000.0
length of validation_samples:  36000.0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_5 (InputLayer)         (None, 6400)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 100)               640100    
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 404       
=================================================================
Total params: 640,504
Trainable params: 640,504
Non-trainable params: 0
_________________________________________________________________
None
"""    

Using TensorFlow backend.


Done loading the libraries
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
sum1:  120000
validation data dimensions:  10114  ,  (10114, 4)
length of train_samples:  84000.0
length of validation_samples:  36000.0
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               640100    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 640,504
Trainable params: 640,504
Non-trainable params: 0
_________________________________________________________________
None
Instruc

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=(array([[0..., epochs=3, steps_per_epoch=120000, verbose=1, validation_steps=0)`


Epoch 1/3
 10217/120000 [=>............................] - ETA: 1:20:31 - loss: 0.2896 - acc: 0.9190

In [6]:
valid_df=pd.read_csv(os.path.join(TEST_DIR, 'test_k{}.csv.gz'.format(0)) )
print(valid_df.columns)
print(Counter(valid_df.y))

Index(['countrycode', 'drawing', 'recognized', 'timestamp', 'word', 'y', 'cv'], dtype='object')
Counter({2: 2557, 0: 2549, 3: 2529, 1: 2479})


In [7]:
from IPython.display import FileLinks
FileLinks('.')

./
  __output__.json
  doodle_trial_model.h5
  __notebook__.ipynb